**Steps:**
1. For each type of model:
    - For each of 5 folds, predict validation set and full test set (OOF: out of fold).
    - Assemble one column with the 5 validation predictions --> to `train_df`
    - Assemble one column with the mean of test predictions --> to `test_df`
2. Concatenate the data from `train_df` and `test_df` with the statistical properties.
3. Do CV as we were doing before on the new data. **Note:** the csv files in the "Final CV" section are three different submissions. Each of them is a "normal" CV.

*****

The files `train_df2.csv` and `test_df2.csv` in the `stacking` directory also include the geometric mean in the statistical properties (see the definition of `stats_prop`). Didn't result in any difference in local CV, but maybe it helps.

In [ ]:
# misc
import warnings
warnings.filterwarnings('ignore')
import gc

# basic imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')
sns.set(palette='colorblind')

# processing
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import VarianceThreshold
from scipy import stats

import shap

# models
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.linear_model import LinearRegression, Ridge

In [ ]:
def stats_prop(df):
    tr_means = []
    tr_medians = []
    tr_stds = []
    tr_skews = []
    tr_mins = []
    tr_maxs = []
    tr_sums = []
    tr_nonzero = []
    tr_prods = []
    for i, row in df.iterrows():
        tr_means.append(row[row.nonzero()[0]].mean())
        tr_medians.append(row[row.nonzero()[0]].median())
        tr_stds.append(row[row.nonzero()[0]].std())
        tr_skews.append(row[row.nonzero()[0]].skew())
        tr_mins.append(row[row.nonzero()[0]].min())
        tr_maxs.append(row[row.nonzero()[0]].max())
        tr_sums.append(row[row.nonzero()[0]].sum())
        tr_nonzero.append(row[row.nonzero()[0]].count()/df.shape[1])
        tr_prods.append(stats.gmean(row[row.nonzero()[0]]))
        
    tr_means = np.nan_to_num(np.array(tr_means))
    tr_medians = np.nan_to_num(np.array(tr_medians))
    tr_stds = np.nan_to_num(np.array(tr_stds))
    tr_skews = np.nan_to_num(np.array(tr_skews))
    tr_mins = np.nan_to_num(np.array(tr_mins))
    tr_maxs = np.nan_to_num(np.array(tr_maxs))
    tr_sums = np.nan_to_num(np.array(tr_sums))
    tr_nonzero = np.nan_to_num(np.array(tr_nonzero))
    tr_prods = np.nan_to_num(np.array(tr_prods))
        
    return np.stack((tr_means, tr_medians, tr_stds, tr_skews, tr_mins, tr_maxs, tr_sums, tr_nonzero, tr_prods), axis=-1)
    

In [ ]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

#### divide by 1000 to overestimate later

In [ ]:
train_data[train_data.columns[1:]] = train_data[train_data.columns[1:]]/1000
test_data[test_data.columns[1:]] = test_data[test_data.columns[1:]]/1000

### Preprocessing

#### Removing target

In [ ]:
# remove target and ID columns
target = train_data.target
train_ID = train_data.ID
train_data = train_data.drop(['target', 'ID'], axis=1)
test_ID = test_data.ID
test_data = test_data.drop(['ID'], axis=1)

#### Variance threshold

In [ ]:
threshold_var = 0.

In [ ]:
vt = VarianceThreshold(threshold=threshold_var)
vt.fit(train_data)
selected_columns = train_data.columns[vt.get_support(indices=True)]

In [ ]:
train_novar = train_data[selected_columns]
test_novar = test_data[selected_columns]

In [ ]:
print('Remaining columns: %i' % len(selected_columns))

#### SHAP

In [ ]:
log_features_train = train_novar
log_features_test = test_novar
log_target = np.log1p(target)

#### Stat all cols

In [ ]:
stat_train = stats_prop(log_features_train)
stat_test = stats_prop(log_features_test)

In [ ]:
# CV folds
kf = KFold(n_splits=10, shuffle=True)
n_splits = kf.get_n_splits()

In [ ]:
# model parameters
params = {'learning_rate':0.5, 'metric':'rmse', 'max_bin':63, 'device':'cpu'}

In [ ]:
shap_values = np.zeros((log_features_train.shape[0],log_features_train.shape[1]+1))

for train_index, test_index in kf.split(log_features_train):
    
    X_tr, X_tst = log_features_train.values[train_index], log_features_train.values[test_index]
    y_tr, y_tst = log_target[train_index], log_target[test_index]
    
    train_set = lgb.Dataset(X_tr, label=y_tr)
    valid_set = lgb.Dataset(X_tst, label=y_tst, reference=train_set)
    
    bst = lgb.train(params, train_set, num_boost_round=50, valid_sets=[train_set, valid_set], early_stopping_rounds=5,
                    verbose_eval=False)

    shap_values += shap.TreeExplainer(bst).shap_values(log_features_train.values)/n_splits

In [ ]:
shap.summary_plot(shap_values, log_features_train, max_display=10, plot_type='dot')

In [ ]:
N = 150
sorted_columns = np.argsort(np.sum(np.abs(shap_values), axis=0)[:-1])[::-1]
most_relevant = train_novar.columns[sorted_columns[:N]]

In [ ]:
log_features_train = log_features_train[most_relevant]
log_features_test = log_features_test[most_relevant]

In [ ]:
print('Remaining columns: %i' % log_features_train.shape[1])

#### Assembly

In [ ]:
train_final = np.hstack((log_features_train.values, stat_train))
test_final = np.hstack((log_features_test.values, stat_test))

In [ ]:
del train_data, test_data, train_novar, test_novar, log_features_train, log_features_test
gc.collect()

In [ ]:
print('Final number of features: %i' % train_final.shape[1])

### oof train+test

In [ ]:
# CV folds
kf = KFold(n_splits=5, shuffle=True, random_state=0)
n_splits = kf.get_n_splits()

In [ ]:
train_df = pd.DataFrame.from_dict({'ID': train_ID, 'xgb': np.zeros(train_ID.shape[0]), 'lgb': np.zeros(train_ID.shape[0]),
                                  'cb': np.zeros(train_ID.shape[0])})
test_df = pd.DataFrame.from_dict({'ID': test_ID, 'xgb': np.zeros(test_ID.shape[0]), 'lgb': np.zeros(test_ID.shape[0]),
                                 'cb': np.zeros(test_ID.shape[0])})

#### xgb

In [ ]:
# model parameters

params = {'booster': 'gbtree', 'learning_rate':0.003, 'colsample_bytree': 0.05, 'eval_metric':'rmse', 'lambda': 3.,
          'alpha': 0.03}

In [ ]:
errors = []

test_set = xgb.DMatrix(test_final)

for train_index, test_index in kf.split(train_final):
        
    X_tr, X_tst = train_final[train_index], train_final[test_index]
    y_tr, y_tst = log_target[train_index], log_target[test_index]

    train_set = xgb.DMatrix(X_tr, label=y_tr)
    valid_set = xgb.DMatrix(X_tst, label=y_tst)

    bst = xgb.train(params, train_set, num_boost_round=20000, evals=[(train_set, 'train'), (valid_set, 'val')],
                    early_stopping_rounds=500, verbose_eval=1000)

    y_val = bst.predict(valid_set, ntree_limit=bst.best_ntree_limit)

    rmsle = np.sqrt(mean_squared_error(y_tst, y_val))

    print('RMSLE: %.5f' % rmsle)

    errors.append(rmsle)
    
    train_df.xgb[test_index] = np.expm1(y_val)

    test_df.xgb = test_df.xgb + np.expm1(bst.predict(test_set, ntree_limit=bst.best_ntree_limit))/n_splits
        
print('\n Fold mean of RMSLE: %.5f' % np.mean(errors))
print('\n Fold std of RMSLE: %.5f' % np.std(errors))

#### lgb

In [ ]:
# model parameters
params = {'boosting': 'gbdt', 'objective':'regression', 'learning_rate':0.01, 'metric':'rmse', 'max_bin':63, 
          'lambda_l2': 0.05, 'device':'cpu', 'feature_fraction': 0.075, 'lambda_l1': 0.01}

In [ ]:
errors = []

for train_index, test_index in kf.split(train_final):
        
    X_tr, X_tst = train_final[train_index], train_final[test_index]
    y_tr, y_tst = log_target[train_index], log_target[test_index]

    train_set = lgb.Dataset(X_tr, label=y_tr)
    valid_set = lgb.Dataset(X_tst, label=y_tst, reference=train_set)

    bst = lgb.train(params, train_set, num_boost_round=20000, valid_sets=[train_set, valid_set], early_stopping_rounds=500, 
                    verbose_eval=1000)

    y_val = bst.predict(X_tst, num_iteration=bst.best_iteration)

    rmsle = np.sqrt(mean_squared_error(y_tst, y_val))

    print('RMSLE: %.5f' % rmsle)

    errors.append(rmsle)
    
    train_df.lgb[test_index] = np.expm1(y_val)
    
    test_df.lgb = test_df.lgb + np.expm1(bst.predict(test_final, num_iteration=bst.best_iteration))/n_splits
        
print('\n Fold mean of RMSLE: %.5f' % np.mean(errors))
print('\n Fold std of RMSLE: %.5f' % np.std(errors))

#### cb

In [ ]:
errors = []

for train_index, test_index in kf.split(train_final):
        
    X_tr, X_tst = train_final[train_index], train_final[test_index]
    y_tr, y_tst = log_target[train_index], log_target[test_index]

    bst = cb.CatBoostRegressor(eta=0.01,iterations=5000, loss_function='RMSE', eval_metric='RMSE', depth=5)
    
    bst.fit(X_tr, y_tr, use_best_model=True, eval_set=(X_tst, y_tst), verbose=True)

    y_val = bst.predict(X_tst)

    rmsle = np.sqrt(mean_squared_error(y_tst, y_val))

    print('RMSLE: %.5f' % rmsle)

    errors.append(rmsle)
    
    train_df.cb[test_index] = np.expm1(y_val)
    
    test_df.cb = test_df.cb + np.expm1(bst.predict(test_final))/n_splits
        
print('\n Fold mean of RMSLE: %.5f' % np.mean(errors))
print('\n Fold std of RMSLE: %.5f' % np.std(errors))

In [ ]:
train_df.to_csv('../submission_files/stacking/train_df_2.csv', index=False)
test_df.to_csv('../submission_files/stacking/test_df_2.csv', index=False)
#train_df = pd.read_csv('../submission_files/stacking/train_df.csv')
#test_df = pd.read_csv('../submission_files/stacking/test_df.csv')

In [ ]:
train_df = train_df.drop(['ID'], axis=1)
test_df = test_df.drop(['ID'], axis=1)

In [ ]:
train_df = train_df.values
test_df = test_df.values

In [ ]:
train_df2 = np.hstack((train_df, stat_train))
test_df2 = np.hstack((test_df, stat_test))

### Final CV

#### cb

In [ ]:
submission = pd.DataFrame.from_dict({'ID': test_ID, 'target': np.zeros(test_ID.shape[0])})

errors = []

for train_index, test_index in kf.split(train_df2):
        
    X_tr, X_tst = train_df2[train_index], train_df2[test_index]
    y_tr, y_tst = log_target[train_index], log_target[test_index]

    bst = cb.CatBoostRegressor(eta=0.1,iterations=200, loss_function='RMSE', eval_metric='RMSE', depth=4)
    
    bst.fit(X_tr, y_tr, use_best_model=True, eval_set=(X_tst, y_tst), verbose=True)

    y_val = bst.predict(X_tst)

    rmsle = np.sqrt(mean_squared_error(y_tst, y_val))

    print('RMSLE: %.5f' % rmsle)

    errors.append(rmsle)
    
    submission.target = submission.target + np.expm1(bst.predict(test_df2))/n_splits
        
print('\n Fold mean of RMSLE: %.5f' % np.mean(errors))
print('\n Fold std of RMSLE: %.5f' % np.std(errors))

#### ceiling to overestimate

In [ ]:
submission.target = 1000*np.ceil(submission.target)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('../submission_files/stacking/cb.csv', index=False)

#### lgb

In [ ]:
# model parameters
params = {'boosting': 'gbdt', 'objective':'regression', 'learning_rate':0.01, 'metric':'rmse', 'max_bin':63, 
          'lambda_l2': 0.05, 'device':'cpu', 'feature_fraction': .44, 'lambda_l1': 0.01}

In [ ]:
submission = pd.DataFrame.from_dict({'ID': test_ID, 'target': np.zeros(test_ID.shape[0])})

errors = []

for train_index, test_index in kf.split(train_df2):
        
    X_tr, X_tst = train_df2[train_index], train_df2[test_index]
    y_tr, y_tst = log_target[train_index], log_target[test_index]

    train_set = lgb.Dataset(X_tr, label=y_tr)
    valid_set = lgb.Dataset(X_tst, label=y_tst, reference=train_set)

    bst = lgb.train(params, train_set, num_boost_round=20000, valid_sets=[train_set, valid_set], early_stopping_rounds=500, 
                    verbose_eval=1000)

    y_val = bst.predict(X_tst, num_iteration=bst.best_iteration)
    
    rmsle = np.sqrt(mean_squared_error(y_tst, y_val))

    print('RMSLE: %.5f' % rmsle)

    errors.append(rmsle)
    
    submission.target = submission.target + np.expm1(bst.predict(test_df2, num_iteration=bst.best_iteration))/n_splits
        
print('\n Fold mean of RMSLE: %.5f' % np.mean(errors))
print('\n Fold std of RMSLE: %.5f' % np.std(errors))

#### ceiling to overestimate

In [ ]:
submission.target = 1000*np.ceil(submission.target)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('../submission_files/stacking/lgb.csv', index=False)

#### xgb

In [ ]:
# model parameters
params = {'booster': 'gbtree', 'learning_rate':0.01, 'colsample_bytree': 0.5, 'eval_metric':'rmse', 'lambda': 3.,
          'alpha': 0.03}

In [ ]:
submission = pd.DataFrame.from_dict({'ID': test_ID, 'target': np.zeros(test_ID.shape[0])})

errors = []

test_set = xgb.DMatrix(test_df2)

for train_index, test_index in kf.split(train_df2):
        
    X_tr, X_tst = train_df2[train_index], train_df2[test_index]
    y_tr, y_tst = log_target[train_index], log_target[test_index]

    train_set = xgb.DMatrix(X_tr, label=y_tr)
    valid_set = xgb.DMatrix(X_tst, label=y_tst)

    bst = xgb.train(params, train_set, num_boost_round=5000, evals=[(train_set, 'train'), (valid_set, 'val')],
                    early_stopping_rounds=500, verbose_eval=1000)

    y_val = bst.predict(valid_set, ntree_limit=bst.best_ntree_limit)

    rmsle = np.sqrt(mean_squared_error(y_tst, y_val))

    print('RMSLE: %.5f' % rmsle)

    errors.append(rmsle)
    
    submission.target = submission.target + np.expm1(bst.predict(test_set, ntree_limit=bst.best_ntree_limit))/n_splits
        
print('\n Fold mean of RMSLE: %.5f' % np.mean(errors))
print('\n Fold std of RMSLE: %.5f' % np.std(errors))

#### ceiling to overestimate

In [ ]:
submission.target = 1000*np.ceil(submission.target)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('../submission_files/stacking/xgb.csv', index=False)